In [1]:
import pandas as pd
from pathlib import Path
import yt_dlp
import numpy as np
from tqdm import tqdm

### 동영상 수집 통합본.csv에 영상 업로드 날짜 추가

In [ ]:
df = pd.read_csv("data_kr/video/동영상 수집 통합본.csv")

upload_dt = []
for url in tqdm(df["url"], total=len(df["url"])):
    if pd.isna(url) or url == '':
        upload_dt.append(None)
        continue

    try:
        ydl_opts = {'quiet': True, 'skip_download': True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            upload_dt.append(info.get("upload_date"))
    except Exception as e:
        upload_dt.append(None)
        print(url)

df["upload_date"] = upload_dt
df["upload_date"] = df["upload_date"].dropna().astype(int).astype(str)
df["upload_date"] = pd.to_datetime(df["upload_date"], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df.to_csv("data_kr/video/동영상 수집 통합본.csv", index=False)

### 업로드 날짜로부터 공시일까지 가격추이 구하기

In [24]:
sectors = ["산업재", "정보기술"]
df = pd.DataFrame()

### LLM 예측 파일 concat ###
for file in Path("preprocessed_data/llm/predict").rglob("*"):
    if file.is_file():
        df_ = pd.read_csv(file)
        df = pd.concat([df, df_], axis=0)
 
### LLM 예측에 실제 등락 라벨 추가 ###
df["code"] = df["code"].astype(str).str.zfill(6)

price_upload = [] # 업로드 당일 종가
price_closure = [] # 공시 당일 종가
for row in tqdm(df.itertuples(), total=len(df)):
    code = str(row.code).zfill(6)
    df_price = pd.read_csv(f"data_kr/price/{code}.csv")
    ### 업로드 날짜 직전 종가
    price_upload.append(df_price.loc[df_price["날짜"] < row.upload_date, "종가"].iloc[-1] if not pd.isna(row.upload_date) else None)
    ### 공시일 날짜 직후 종가
    price_closure.append(df_price.loc[df_price["날짜"] >= row.disclosure_date, "종가"].iloc[0] if not pd.isna(row.disclosure_date) else None)

df["price_upload"] = price_upload
df["price_closure"] = price_closure
label = df["price_upload"] < df["price_closure"] # (업로드 당일 종가)보다 (공시 당일 종가)가 올랐는가?
df["label"] = ["up" if l == True else "down" for l in label]
df["prediction_match"] = df["prediction"] == df["label"]

df = df[["code", "name", "sector", "year", "quarter", "upload_date", "disclosure_date", "price_upload", "price_closure", "label", "prediction", "prediction_match"]]
df.to_csv("preprocessed_data/llm/predict/업로드_공시_가격비교.csv", index=False)

100%|█████████████████████████████████████████████████████████████████████████████| 2940/2940 [00:10<00:00, 291.96it/s]
